# Limpieza de Datos 

In [280]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
import json
data_path = '../../data/'

# Índice de complejidad económica por alcaldía
Extrar el índice de complejidad económica por cada alcaldía de la Ciudad de México. El ECI por alcaldía se obtuvo de https://datamexico.org

## Lista de alcaldías de la CDMX

In [281]:
alc_df = pd.read_csv(data_path + 'alcaldias_cdmx.csv')
alc_df.loc[4] = 'Cuajimalpa de Morelos'
alc_df.loc[9] = 'La Magdalena Contreras'
alcaldias = list(alc_df.alcaldia)
print(alcaldias)

['Álvaro Obregón', 'Azcapotzalco', 'Benito Juárez', 'Coyoacán', 'Cuajimalpa de Morelos', 'Cuauhtémoc', 'Gustavo A. Madero', 'Iztacalco', 'Iztapalapa', 'La Magdalena Contreras', 'Miguel Hidalgo', 'Milpa Alta', 'Tláhuac', 'Tlalpan', 'Venustiano Carranza', 'Xochimilco']


## Obteniendo los ECI por municipio
El dataframe ```municipios_eci_df``` contiene los índices de complejidad económica con un nivel geográfico de municipio y un nivel industrial basado en el SCIAN para el año 2020 y bajo un indicador de empleados.

In [282]:
eci_url = "https://datamexico.org/api/stats/eci?cube=inegi_denue&Month=20200417,20191114,20190410&rca=Municipality,NAICS%20Industry,Number%20of%20Employees%20Midpoint&threshold=NAICS%20Industry:900,Municipality:900&debug=true&locale=es"
municipios_eci_df = pd.read_json(eci_url)
municipios_eci_df = pd.DataFrame(municipios_eci_df.data.values.tolist())

In [283]:
municipios_eci_df.rename(columns={'Municipality ID': 'municipio_id', 'Number of Employees Midpoint ECI':'ECI', 'Municipality':'municipio'}, inplace=True)
del municipios_eci_df['municipio_id']
municipios_eci_df.head()

,ECI,municipio
0,3.049991,Aguascalientes
1,-0.361775,Asientos
2,0.342401,Calvillo
3,-0.513439,Cosío
4,3.094544,Jesús María


## Filtrando por alcaldías de la CDMX 

In [284]:
alcaldias_eci_df = municipios_eci_df[municipios_eci_df.municipio.isin(alcaldias)]

In [285]:
alcaldias_dict = dict()
for _, row in alcaldias_eci_df.iterrows():
    municipio = row.municipio
    eci = row.ECI
    if municipio not in alcaldias_dict:
        alcaldias_dict[municipio] = [-10e2]
    alcaldias_dict[municipio] = [max(eci, alcaldias_dict[municipio][0])]

alcaldias_names = alcaldias_dict.keys()
eci_values = []
for alcaldia in alcaldias_names:
    eci_values.append(alcaldias_dict[alcaldia][0])

In [286]:
alcaldias_eci_clean_df = pd.DataFrame(data={'alcaldia': alcaldias_names, 'eci': eci_values})
alcaldias_eci_clean_df.loc[0, 'alcaldia'] = 'Cuauhtemoc'
alcaldias_eci_clean_df.loc[3, 'alcaldia'] = 'Coyoacan'
alcaldias_eci_clean_df.loc[10, 'alcaldia'] = 'Alvaro Obregon'
alcaldias_eci_clean_df.loc[11, 'alcaldia'] = 'Tlahuac'
alcaldias_eci_clean_df.loc[14, 'alcaldia'] = 'Benito Juarez'
alcaldias_eci_clean_df

,alcaldia,eci
0,Cuauhtemoc,3.471986
1,Venustiano Carranza,2.501312
2,Azcapotzalco,3.865509
3,Coyoacan,2.800526
4,Cuajimalpa de Morelos,3.854617
5,Gustavo A. Madero,2.680508
6,Iztacalco,2.944861
7,Iztapalapa,2.629184
8,La Magdalena Contreras,1.623270
9,Milpa Alta,0.343307


In [287]:
alcaldias_eci_clean_df.alcaldia = alcaldias_eci_clean_df.alcaldia.str.lower()
alcaldias_eci_clean_df

,alcaldia,eci
0,cuauhtemoc,3.471986
1,venustiano carranza,2.501312
2,azcapotzalco,3.865509
3,coyoacan,2.800526
4,cuajimalpa de morelos,3.854617
5,gustavo a. madero,2.680508
6,iztacalco,2.944861
7,iztapalapa,2.629184
8,la magdalena contreras,1.623270
9,milpa alta,0.343307


In [288]:
alcaldias_eci_clean_df.to_csv(data_path + 'alcaldias_eci.csv', index=False)

# Colonias prioritarias CDMX

El archivo ```colonias_prioritarias.csv``` contiene una lista de las 200 colonias de la Ciudad de México con mayor prioridad de atención médica según el gobierno de la ciudad. Este archivo fue extraido de: https://datos.cdmx.gob.mx/dataset/colonias-de-atencion-prioritaria-covid-kioscos

In [289]:
colonias_df = pd.read_csv(data_path + 'colonias_prioritarias.csv')
colonias_df.columns = colonias_df.columns.str.lower()
colonias_df.columns = colonias_df.columns.str.replace(' ', '_')
colonias_df.drop(columns=['tasa_activos_(100mil_habitantes)','geo_shape', 'clave_colonia', 'centroide_colonia'], inplace=True)
colonias_df.rename(columns={'alcaldía': 'alcaldia', 'población_total':'poblacion_total'}, inplace=True)
colonias_df.colonia = colonias_df.colonia.str.lower()
colonias_df.alcaldia = colonias_df.alcaldia.str.lower()
colonias_df.porcentaje_activos = colonias_df.porcentaje_activos*100

In [290]:
colonias_df.head()

,alcaldia,colonia,casos_activos,poblacion_total,porcentaje_activos
0,alvaro obregon,barrio norte,57,14367.414210,0.2562
1,alvaro obregon,jalalpa el grande,47,10972.245610,0.2112
2,alvaro obregon,jardines del pedregal,39,13817.000000,0.1753
3,alvaro obregon,merced gomez,48,6992.834615,0.2157
4,alvaro obregon,olivar del conde 2da seccion ii,37,8191.441839,0.1663


### Identificación y corrección de valores nulos

In [291]:
colonias_df.isna().sum()

alcaldia              0
colonia               0
casos_activos         0
poblacion_total       2
porcentaje_activos    0
dtype: int64

In [292]:
colonias_df[colonias_df.poblacion_total.isna()]

,alcaldia,colonia,casos_activos,poblacion_total,porcentaje_activos
47,tlahuac,la conchita i,22,NaN,0.0989
111,tlahuac,la conchita ii,102,NaN,0.4584


In [293]:
colonias_df.loc[47, 'poblacion_total'] = (colonias_df.loc[47].casos_activos * 100) / colonias_df.loc[47].porcentaje_activos
colonias_df.loc[111, 'poblacion_total'] = (colonias_df.loc[111].casos_activos * 100) / colonias_df.loc[111].porcentaje_activos

In [294]:
colonias_df.isna().sum()

alcaldia              0
colonia               0
casos_activos         0
poblacion_total       0
porcentaje_activos    0
dtype: int64

## Agregar código postal a las colonias

In [295]:
seen = set()
repeated = []
for col in list(colonias_df.colonia):
    if col in seen:
        repeated.append(col)
        continue
    seen.add(col)
repeated

['morelos ii', 'miguel hidalgo']

In [296]:
colonias_df[colonias_df.colonia.isin(repeated)]

,alcaldia,colonia,casos_activos,poblacion_total,porcentaje_activos
54,tlalpan,miguel hidalgo,65,13276.78689,0.2921
63,venustiano carranza,morelos ii,38,17867.00000,0.1708
158,cuauhtemoc,morelos ii,18,12171.00000,0.0809
182,tlahuac,miguel hidalgo,43,13122.58258,0.1933


In [297]:
colonias_df.loc[54, 'colonia'] = 'miguel hidalgo - tlal'
colonias_df.loc[63, 'colonia'] = 'morelos ii - vc'
colonias_df.loc[158, 'colonia'] = 'morelos ii - cu'
colonias_df.loc[182, 'colonia'] = 'miguel hidalgo - tlah'

In [298]:
colonias_df['cp'] = np.nan

In [299]:
colonias_df.head()

,alcaldia,colonia,casos_activos,poblacion_total,porcentaje_activos,cp
0,alvaro obregon,barrio norte,57,14367.414210,0.2562,NaN
1,alvaro obregon,jalalpa el grande,47,10972.245610,0.2112,NaN
2,alvaro obregon,jardines del pedregal,39,13817.000000,0.1753,NaN
3,alvaro obregon,merced gomez,48,6992.834615,0.2157,NaN
4,alvaro obregon,olivar del conde 2da seccion ii,37,8191.441839,0.1663,NaN


In [300]:
cp_df = pd.read_csv(data_path + 'codigos_postales.csv')
cp_df.head()

,CP,Estado,Municipio,Tipo de Asentamiento,Asentamiento,Clave de Oficina
0,9830,Ciudad de México,Iztapalapa,Colonia,Paraje San Juan,9821
1,9837,Ciudad de México,Iztapalapa,Colonia,San Miguel 8va. Ampliación,9821
2,9838,Ciudad de México,Iztapalapa,Colonia,Plan de Iguala,9821
3,9839,Ciudad de México,Iztapalapa,Colonia,Ampliación Paraje San Juan,9821
4,9839,Ciudad de México,Iztapalapa,Colonia,San Juan Joya,9821


In [301]:
cp_df.Asentamiento = cp_df.Asentamiento.str.lower()
cp_df.head()

,CP,Estado,Municipio,Tipo de Asentamiento,Asentamiento,Clave de Oficina
0,9830,Ciudad de México,Iztapalapa,Colonia,paraje san juan,9821
1,9837,Ciudad de México,Iztapalapa,Colonia,san miguel 8va. ampliación,9821
2,9838,Ciudad de México,Iztapalapa,Colonia,plan de iguala,9821
3,9839,Ciudad de México,Iztapalapa,Colonia,ampliación paraje san juan,9821
4,9839,Ciudad de México,Iztapalapa,Colonia,san juan joya,9821


In [302]:
from collections import Counter
def should_be_equal(word1, word2, per=0.4):
    s1 = word1.lower().replace(' ', '').replace('(', '').replace(')', '').replace('.','')
    s2 = word2.lower().replace(' ', '').replace('(', '').replace(')', '').replace('.','')
    max_len = max(len(s1), len(s2))
    sum_ = 0
    if s1[0] == s2[0]:
        for i in range(min(len(s1), len(s2))):
            if s1[i] == s2[i]:
                sum_ += 1
    #print(sum_, len(s1), len(s2), max_len, s1, s2, int(max_len*per))
    if max_len - sum_ <= int(max_len*per):
        return True 
    return False

In [303]:
found_col = dict()
for _, rowCol in colonias_df.iterrows():
    colonia = rowCol.colonia
    col_alcaldia = rowCol.alcaldia
    if colonia not in found_col:
        found_col[(colonia, col_alcaldia)] = []
    for _, row in cp_df.iterrows():
        asentamiento = row.Asentamiento
        cp = row.CP
        alcaldia = row.Municipio
        if should_be_equal(colonia, asentamiento) and len(col_alcaldia) == len(alcaldia):
            found_col[(colonia, col_alcaldia)].append((asentamiento, alcaldia, cp))

In [304]:
for _, row in colonias_df.iterrows():
    tup = (row.colonia, row.alcaldia)
    if tup in found_col.keys():
        if len(found_col[tup]) > 0:
            colonias_df.loc[colonias_df.colonia == row.colonia, 'cp'] = str(int(found_col[tup][0][2]))

In [305]:
colonias_df.loc[57	 , 'cp'] = '14250'
colonias_df.loc[191 , 'cp'] = '14250'
colonias_df.loc[38	 , 'cp'] = '11320'
colonias_df.loc[108 , 'cp'] = '11320'
colonias_df.loc[127 , 'cp'] = '16090'
colonias_df.loc[94	 , 'cp'] = '7960'
colonias_df.loc[64	 , 'cp'] = '15640'
colonias_df.loc[40	 , 'cp'] = '11800'
colonias_df.loc[121 , 'cp'] = '15000'
colonias_df.loc[47	 , 'cp'] = '13360'
colonias_df.loc[111 , 'cp'] = '13360'
colonias_df.loc[95	 , 'cp'] = '7140'
colonias_df.loc[100 , 'cp'] = '9910'
colonias_df.loc[158 , 'cp'] = '6200'
colonias_df.loc[63	 , 'cp'] = '15270'
colonias_df.loc[143 , 'cp'] = '3600'
colonias_df.loc[80	 , 'cp'] = '5219'
colonias_df.loc[162 , 'cp'] = '8100'
colonias_df.loc[56	 , 'cp'] = '14100'
colonias_df.loc[59	 , 'cp'] = '14100'
colonias_df.loc[61	 , 'cp'] = '14100'
colonias_df.loc[79	 , 'cp'] = '4369'
colonias_df.loc[10	 , 'cp'] = '4369'
colonias_df.loc[76	 , 'cp'] = '4369'
colonias_df.loc[146 , 'cp'] = '4369'
colonias_df.loc[13	 , 'cp'] = '4369'
colonias_df.loc[149 , 'cp'] = '4369'
colonias_df.loc[51	 , 'cp'] = '13099'
colonias_df.loc[42	 , 'cp'] = '12100'
colonias_df.loc[173 , 'cp'] = '10400'
colonias_df.loc[114 , 'cp'] = '13000'
colonias_df.loc[46	 , 'cp'] = '13310'
colonias_df.loc[43	 , 'cp'] = '12400'
colonias_df.loc[110 , 'cp'] = '12200'
colonias_df.loc[180 , 'cp'] = '13070'
colonias_df.loc[194 , 'cp'] = '16700'
colonias_df.loc[181 , 'cp'] = '13300'
colonias_df.loc[50	 , 'cp'] = '13420'
colonias_df.loc[140 , 'cp'] = '3310'
colonias_df.loc[199 , 'cp'] = '16029'
colonias_df.loc[129 , 'cp'] = '1090'
colonias_df.loc[184 , 'cp'] = '13300'

In [306]:
colonias_df.head()

,alcaldia,colonia,casos_activos,poblacion_total,porcentaje_activos,cp
0,alvaro obregon,barrio norte,57,14367.414210,0.2562,1410
1,alvaro obregon,jalalpa el grande,47,10972.245610,0.2112,1377
2,alvaro obregon,jardines del pedregal,39,13817.000000,0.1753,1900
3,alvaro obregon,merced gomez,48,6992.834615,0.2157,1600
4,alvaro obregon,olivar del conde 2da seccion ii,37,8191.441839,0.1663,1408


In [307]:
col

'tepepan (ampl)'

## Guardar dataframe limpio

In [308]:
colonias_df.to_csv(data_path + 'colonias_prioritarias_clean.csv', index=False)

In [309]:
colonias_df.head()

,alcaldia,colonia,casos_activos,poblacion_total,porcentaje_activos,cp
0,alvaro obregon,barrio norte,57,14367.414210,0.2562,1410
1,alvaro obregon,jalalpa el grande,47,10972.245610,0.2112,1377
2,alvaro obregon,jardines del pedregal,39,13817.000000,0.1753,1900
3,alvaro obregon,merced gomez,48,6992.834615,0.2157,1600
4,alvaro obregon,olivar del conde 2da seccion ii,37,8191.441839,0.1663,1408


# Negocios CDMX
Esta lista contiene los negocios dentro de la Ciudad de México que solicitaron financiamiento dentro del programa: Financiamiento para Microempresas del Fondo para el Desarrollo Social de la Ciudad de México. 

La base de datos original se encuentra en: https://datos.cdmx.gob.mx/dataset/solicitudes-financiamiento-microempresas

In [310]:
negocios_df = pd.read_csv(data_path + 'microempresas.csv')

In [311]:
def lower_cols_rows(df):
    df.columns = df.columns.str.lower()
    for column in df.columns:
        df[str(column)] = df[str(column)].str.lower()
    return df

In [312]:
negocios_df.drop(columns=['SEXO', 'FOLIO', 'ESTADO_DE_NACIMIENTO', 'FECHA_INICIO_SOLICITUD', 'FECHA_TERMINO'], inplace=True)
negocios_df = lower_cols_rows(negocios_df)

In [313]:
negocios_df.head()

,ocupacion,alcaldia,c_p,colonia,giro,num_empleados,estatus
0,joyeria�,alvaro obregon,1260,la mexicana,joyeria�,0,cobrado
1,tiendas de ropa�,alvaro obregon,1260,calzada jalalpa,tiendas de ropa�,3,cobrado
2,tiendas de abarrotes�,alvaro obregon,1780,olivar de los padres,tiendas de abarrotes�,0,cobrado
3,tiendas de abarrotes�,alvaro obregon,1780,olivar de los padres,tiendas de abarrotes�,3,cobrado
4,tiendas de abarrotes�,alvaro obregon,1780,olivar de los padres,tiendas de abarrotes�,0,cobrado


In [314]:
negocios_df = negocios_df[negocios_df.estatus == 'cobrado']
negocios_df.drop(columns=['estatus'], inplace=True)
negocios_df.shape

(51198, 6)

In [315]:
negocios_df.isna().sum()

ocupacion          9
alcaldia           0
c_p                1
colonia           11
giro             128
num_empleados      0
dtype: int64

### Ocupación

In [316]:
negocios_df[negocios_df.ocupacion.isna()]

,ocupacion,alcaldia,c_p,colonia,giro,num_empleados
117391,NaN,venustiano carranza,15290,10 de mayo,juegos mecanicos feria,1
118507,NaN,cuauhtemoc,6220,peralvillo,panaderia,1
118524,NaN,azcapotzalco,2630,coltongo,purificadora de agua,1
118532,NaN,iztapalapa,9200,unidad habitacional vicente guerrero,comercio,1
118537,NaN,iztacalco,8910,barrio santa cruz,artesanias y juegos didacticos,1
118542,NaN,iztapalapa,9200,unidad habitacional vicente guerrero,comercio,1
118908,NaN,cuauhtemoc,6720,doctores,"promotora deportiva gama soccer, ac",1
118923,NaN,iztapalapa,9099,heroes de churubusco,liga de futbol siglo xxi plus a.c,1
118924,NaN,iztapalapa,9060,escuadron 201,escuela de futbol pumas 201,1


In [317]:
indices = list(negocios_df[negocios_df.ocupacion.isna()].index)
for index in indices:
    negocios_df.loc[index, 'ocupacion'] = negocios_df.loc[index, 'giro']

In [318]:
negocios_df[negocios_df.index.isin(indices)]

,ocupacion,alcaldia,c_p,colonia,giro,num_empleados
117391,juegos mecanicos feria,venustiano carranza,15290,10 de mayo,juegos mecanicos feria,1
118507,panaderia,cuauhtemoc,6220,peralvillo,panaderia,1
118524,purificadora de agua,azcapotzalco,2630,coltongo,purificadora de agua,1
118532,comercio,iztapalapa,9200,unidad habitacional vicente guerrero,comercio,1
118537,artesanias y juegos didacticos,iztacalco,8910,barrio santa cruz,artesanias y juegos didacticos,1
118542,comercio,iztapalapa,9200,unidad habitacional vicente guerrero,comercio,1
118908,"promotora deportiva gama soccer, ac",cuauhtemoc,6720,doctores,"promotora deportiva gama soccer, ac",1
118923,liga de futbol siglo xxi plus a.c,iztapalapa,9099,heroes de churubusco,liga de futbol siglo xxi plus a.c,1
118924,escuela de futbol pumas 201,iztapalapa,9060,escuadron 201,escuela de futbol pumas 201,1


In [319]:
negocios_df.isna().sum()

ocupacion          0
alcaldia           0
c_p                1
colonia           11
giro             128
num_empleados      0
dtype: int64

### Colonia

In [320]:
negocios_df[negocios_df.colonia.isna()]

,ocupacion,alcaldia,c_p,colonia,giro,num_empleados
38833,comerciante,coyoacan,4470,NaN,tienda de abarrotes,2
38950,comerciante,coyoacan,4330,NaN,ferreteria,4
38991,comerciante,tlahuac,13300,NaN,cocina,1
40360,comerciante,venustiano carranza,15290,NaN,alimentos,2
40361,comerciante,tlalpan,14738,NaN,renta de equipo de computo,1
118074,comerciante,xochimilco,16090,NaN,cocina,3
118123,comerciante,tlahuac,13200,NaN,"venta de fragancias, escncias y perfumes",1
118195,comerciante,tlalpan,14430,NaN,cafeteria,1
118203,comerciante,coyoacan,4480,NaN,venta de ropa y accesorios,1
118289,comerciante,xochimilco,16090,NaN,panaderia,2


In [321]:
indices = list(negocios_df[negocios_df.colonia.isna()].index)

In [322]:
negocios_df.loc[38833, 'colonia'] = 'Presidentes Ejidales 1a Seccion'
negocios_df.loc[38950, 'colonia'] = 'El Rosedal'
negocios_df.loc[38991, 'colonia'] = 'Santa Ana Centro'
negocios_df.loc[40360, 'colonia'] = '10 de Mayo '
negocios_df.loc[40361, 'colonia'] = 'Bosques del Pedregal'
negocios_df.loc[118074, 'colonia'] = 'Tablas de San Lorenzo'
negocios_df.loc[118123, 'colonia'] = 'Miguel Hidalgo'
negocios_df.loc[118195, 'colonia'] = 'Tlalcoligia'
negocios_df.loc[118203, 'colonia'] = 'Culhuacan CTM CROC'
negocios_df.loc[118289, 'colonia'] = 'Tablas de San Lorenzo'
negocios_df.loc[118585, 'colonia'] = 'El Rosario'

In [323]:
negocios_df[negocios_df.index.isin(indices)]

,ocupacion,alcaldia,c_p,colonia,giro,num_empleados
38833,comerciante,coyoacan,4470,Presidentes Ejidales 1a Seccion,tienda de abarrotes,2
38950,comerciante,coyoacan,4330,El Rosedal,ferreteria,4
38991,comerciante,tlahuac,13300,Santa Ana Centro,cocina,1
40360,comerciante,venustiano carranza,15290,10 de Mayo,alimentos,2
40361,comerciante,tlalpan,14738,Bosques del Pedregal,renta de equipo de computo,1
118074,comerciante,xochimilco,16090,Tablas de San Lorenzo,cocina,3
118123,comerciante,tlahuac,13200,Miguel Hidalgo,"venta de fragancias, escncias y perfumes",1
118195,comerciante,tlalpan,14430,Tlalcoligia,cafeteria,1
118203,comerciante,coyoacan,4480,Culhuacan CTM CROC,venta de ropa y accesorios,1
118289,comerciante,xochimilco,16090,Tablas de San Lorenzo,panaderia,2


In [324]:
negocios_df.isna().sum()

ocupacion          0
alcaldia           0
c_p                1
colonia            0
giro             128
num_empleados      0
dtype: int64

### Giro

In [325]:
indices = list(negocios_df[negocios_df.giro.isna()].index)
for index in indices:
    negocios_df.loc[index, 'giro'] = negocios_df.loc[index, 'ocupacion']

In [326]:
negocios_df[negocios_df.index.isin(indices)]

,ocupacion,alcaldia,c_p,colonia,giro,num_empleados
8676,autoempleda,iztapalapa,9230,jos maria morelos y pavon,autoempleda,1
10342,comerciante,azcapotzalco,2400,san juan tlihuaca,comerciante,2
10349,comerciante,iztapalapa,9840,los reyes culhuacn,comerciante,1
10505,comerciante,alvaro obregon,1830,santa rosa xochiac,comerciante,2
11029,comerciante,cuauhtemoc,6860,vista alegre,comerciante,2
...,...,...,...,...,...,...
86127,restaurantero,iztapalapa,9230,jos maria morelos y pavon,restaurantero,3
86174,estilista,tlahuac,13430,ampliacion selene,estilista,3
86391,comerciante,xochimilco,16038,ampliacion san marcos norte,comerciante,1
87050,"albañil , pintor y plomero",iztacalco,8710,tlazintla,"albañil , pintor y plomero",2


In [327]:
negocios_df.isna().sum()

ocupacion        0
alcaldia         0
c_p              1
colonia          0
giro             0
num_empleados    0
dtype: int64

### Obteniendo únicamente negocios en las 200 colonias prioritarias

In [328]:
negocios_df.colonia = negocios_df.colonia.str.lower()

In [329]:
negocios_df.head()

,ocupacion,alcaldia,c_p,colonia,giro,num_empleados
0,joyeria�,alvaro obregon,1260,la mexicana,joyeria�,0
1,tiendas de ropa�,alvaro obregon,1260,calzada jalalpa,tiendas de ropa�,3
2,tiendas de abarrotes�,alvaro obregon,1780,olivar de los padres,tiendas de abarrotes�,0
3,tiendas de abarrotes�,alvaro obregon,1780,olivar de los padres,tiendas de abarrotes�,3
4,tiendas de abarrotes�,alvaro obregon,1780,olivar de los padres,tiendas de abarrotes�,0


In [330]:
negocios_df.shape

(51198, 6)

In [331]:
negocios_cp = set(negocios_df.c_p.unique())
colonias_cp = list(colonias_df.cp)
seen = []
for cp in colonias_cp:
    if cp in negocios_cp:
        seen.append(cp)
negocios_df = negocios_df[negocios_df.c_p.isin(seen)]

In [332]:
negocios_df.head()

,ocupacion,alcaldia,c_p,colonia,giro,num_empleados
0,joyeria�,alvaro obregon,1260,la mexicana,joyeria�,0
1,tiendas de ropa�,alvaro obregon,1260,calzada jalalpa,tiendas de ropa�,3
2,tiendas de abarrotes�,alvaro obregon,1780,olivar de los padres,tiendas de abarrotes�,0
3,tiendas de abarrotes�,alvaro obregon,1780,olivar de los padres,tiendas de abarrotes�,3
4,tiendas de abarrotes�,alvaro obregon,1780,olivar de los padres,tiendas de abarrotes�,0


In [333]:
cp_col_dict = dict()
for _, row in colonias_df.iterrows():
    colonia = row.colonia
    cp = row.cp
    if cp not in cp_col_dict:
        cp_col_dict[cp] = []
    cp_col_dict[cp].append(colonia)
for c in cp_col_dict.keys():
    print(f'{c}: \n\t {cp_col_dict[c]}')

1410: 
	 ['barrio norte']
1377: 
	 ['jalalpa el grande']
1900: 
	 ['jardines del pedregal']
1600: 
	 ['merced gomez']
1408: 
	 ['olivar del conde 2da seccion ii']
1290: 
	 ['piloto (adolfo lopez mateos)', 'piloto (adolfo lopez mateos) (ampl)', 'presidentes']
1400: 
	 ['olivar del conde 1ra seccion i', 'olivar del conde 1ra seccion ii']
3660: 
	 ['san simon ticumac']
3300: 
	 ['portales i', 'portales iii']
4369: 
	 ['pedregal de sto domingo iii', 'pedregal de santa ursula i', 'pedregal de sto domingo vi', 'pedregal de santa ursula iv', 'pedregal de sto domingo iv', 'pedregal de santa ursula ii', 'pedregal de sto domingo ii', 'pedregal de sto domingo v', 'pedregal de sto domingo vii']
4600: 
	 ['pedregal de santa ursula iii']
12400: 
	 ['centro iv', 'centro vi', 'centro viii', 'san pablo oztotepec (pblo)']
16450: 
	 ['santa maria la ribera iv', 'santa maria la ribera i', 'santa maria la ribera ii', 'santa maria nativitas (pblo)']
6450: 
	 ['atlampa']
6820: 
	 ['transito']
6720: 
	 ['doct

In [334]:
colonias_to_drop = set()
for index, row in negocios_df.iterrows():
    cp = row.c_p
    colonia = row.colonia.lower()
    if cp in cp_col_dict:
        found = False
        for real_col in cp_col_dict[cp]:
            if should_be_equal(colonia, real_col):
                found = True
        if not found:
            colonias_to_drop.add(colonia)

In [335]:
negocios_df = negocios_df[~negocios_df.colonia.isin(colonias_to_drop)]

## Guardar dataframe limpio

In [336]:
negocios_df.to_csv(data_path + 'microempresas_clean.csv', index=False)

In [337]:
negocios_df.head()

,ocupacion,alcaldia,c_p,colonia,giro,num_empleados
1,tiendas de ropa�,alvaro obregon,1260,calzada jalalpa,tiendas de ropa�,3
2,tiendas de abarrotes�,alvaro obregon,1780,olivar de los padres,tiendas de abarrotes�,0
3,tiendas de abarrotes�,alvaro obregon,1780,olivar de los padres,tiendas de abarrotes�,3
4,tiendas de abarrotes�,alvaro obregon,1780,olivar de los padres,tiendas de abarrotes�,0
6,venta de zapatos y productos de belleza,alvaro obregon,1290,presidentes,venta de zapatos y productos de belleza,0


In [338]:
negocios_df.shape

(10747, 6)